#### To check nitty gritties of data I converted to excel to have a look

In [6]:
import pandas as pd

file_path = "/Users/pranavlakhotia/Downloads/data.parquet"
df = pd.read_parquet(file_path)

df_reset = df.reset_index()

datetime_cols = [col for col in df_reset.columns if pd.api.types.is_datetime64_any_dtype(df_reset[col])]

if datetime_cols:
    # Take the first datetime column (assuming it's your timestamp index)
    ts_col = datetime_cols[0]
    df_reset[ts_col] = df_reset[ts_col].dt.strftime('%Y-%m-%d %H:%M:%S')
    df_reset.rename(columns={ts_col: 'Timestamp'}, inplace=True)
    print(f"🕒 Timestamp column detected and formatted: {ts_col}")
else:
    print("⚠️ No datetime column detected — check the data manually.")

# Save to Excel
output_path = "/Users/pranavlakhotia/Downloads/nifty_banknifty_data.xlsx"
df_reset.to_excel(output_path, index=False)

print(f"✅ File successfully saved as: {output_path}")

🕒 Timestamp column detected and formatted: time
✅ File successfully saved as: /Users/pranavlakhotia/Downloads/nifty_banknifty_data.xlsx


In [8]:
df_raw = df.copy()

#### Filtered the dataset to retain only trading hours (9:15 AM – 3:30 PM) for each day between Jan 2021 and Jun 2022, dropping all non-market-hour records where prices remained static after market close.

In [42]:
df = df_raw

df = df.reset_index()

df['time'] = pd.to_datetime(df['time'])

market_df = df[(df['time'].dt.time >= pd.to_datetime('09:15:00').time()) &
               (df['time'].dt.time <= pd.to_datetime('15:30:00').time())]

market_df

,time,banknifty,nifty,tte
0,2021-01-01 09:15:00,0.286058,0.199729,27
1,2021-01-01 09:16:00,0.285381,0.200433,27
2,2021-01-01 09:17:00,0.284233,0.200004,27
3,2021-01-01 09:18:00,0.286104,0.199860,27
4,2021-01-01 09:19:00,0.285539,0.198951,27
...,...,...,...,...
690507,2022-06-30 15:26:00,0.240701,0.214758,28
690508,2022-06-30 15:27:00,0.240875,0.216558,28
690509,2022-06-30 15:28:00,0.242115,0.216794,28
690510,2022-06-30 15:29:00,0.243426,0.216455,28


In [43]:
market_df[market_df['banknifty'].isna()]

,time,banknifty,nifty,tte
4570,2021-01-04 13:25:00,NaN,0.184289,24
7495,2021-01-06 14:10:00,NaN,0.195641,22
8687,2021-01-07 10:02:00,NaN,0.175629,21
8816,2021-01-07 12:11:00,NaN,0.179667,21
11526,2021-01-09 09:21:00,NaN,0.173267,20
...,...,...,...,...
680353,2022-06-23 14:12:00,NaN,0.208540,7
681604,2022-06-24 11:03:00,NaN,0.204001,6
687380,2022-06-28 11:19:00,NaN,0.206868,30
687432,2022-06-28 12:11:00,NaN,0.206120,30


#### For every missing banknifty value, we filled it using a weighted average of the previous and next 5-minute valid data points, giving higher weight to closer timestamps. The window dynamically expanded (up to ±10 minutes) if nearby data was missing, ensuring stable interpolation even near gaps.

In [44]:
def weighted_fill(series, time_col, target_col):
    df = series.copy()
    weights = np.array([5,4,3,2,1,1,2,3,4,5])  # more weight to nearer timestamps
    weights = weights / weights.sum()  # normalize

    for i in df.index[df[target_col].isna()]:
        t = df.loc[i, time_col]
        
        # Create ±5 minute window
        window = df[(df[time_col] >= t - pd.Timedelta(minutes=8)) & 
                    (df[time_col] <= t + pd.Timedelta(minutes=8))].dropna(subset=[target_col])
        
        # Adjust dynamically if less than 10 valid points
        if len(window) < 10:
            window = df[(df[time_col] >= t - pd.Timedelta(minutes=10)) & 
                        (df[time_col] <= t + pd.Timedelta(minutes=10))].dropna(subset=[target_col])
        
        # If still no valid window, skip
        if len(window) == 0:
            continue
        
        # Sort by time and use weighted mean
        window = window.sort_values(time_col)
        valid_len = min(len(window), 10)
        w = weights[:valid_len]
        w = w / w.sum()
        df.loc[i, target_col] = np.average(window[target_col].iloc[:valid_len], weights=w)

    return df

In [45]:
import numpy as np

market_df = weighted_fill(market_df, 'time', 'banknifty')

market_df

,time,banknifty,nifty,tte
0,2021-01-01 09:15:00,0.286058,0.199729,27
1,2021-01-01 09:16:00,0.285381,0.200433,27
2,2021-01-01 09:17:00,0.284233,0.200004,27
3,2021-01-01 09:18:00,0.286104,0.199860,27
4,2021-01-01 09:19:00,0.285539,0.198951,27
...,...,...,...,...
690507,2022-06-30 15:26:00,0.240701,0.214758,28
690508,2022-06-30 15:27:00,0.240875,0.216558,28
690509,2022-06-30 15:28:00,0.242115,0.216794,28
690510,2022-06-30 15:29:00,0.243426,0.216455,28


In [46]:
market_df[market_df['banknifty'].isna()]

,time,banknifty,nifty,tte


#### Similarly filling null data points for nifty

In [47]:
market_df[market_df['nifty'].isna()]

,time,banknifty,nifty,tte
204,2021-01-01 12:39:00,0.256850,NaN,27
1643,2021-01-02 12:38:00,0.255651,NaN,27
4398,2021-01-04 10:33:00,0.258322,NaN,24
5969,2021-01-05 12:44:00,0.262732,NaN,23
7363,2021-01-06 11:58:00,0.262900,NaN,22
...,...,...,...,...
684465,2022-06-26 10:44:00,0.222915,NaN,6
687532,2022-06-28 13:51:00,0.235283,NaN,30
688901,2022-06-29 12:40:00,0.235789,NaN,29
689022,2022-06-29 14:41:00,0.236893,NaN,29


In [48]:
market_df = weighted_fill(market_df, 'time', 'nifty')

market_df

,time,banknifty,nifty,tte
0,2021-01-01 09:15:00,0.286058,0.199729,27
1,2021-01-01 09:16:00,0.285381,0.200433,27
2,2021-01-01 09:17:00,0.284233,0.200004,27
3,2021-01-01 09:18:00,0.286104,0.199860,27
4,2021-01-01 09:19:00,0.285539,0.198951,27
...,...,...,...,...
690507,2022-06-30 15:26:00,0.240701,0.214758,28
690508,2022-06-30 15:27:00,0.240875,0.216558,28
690509,2022-06-30 15:28:00,0.242115,0.216794,28
690510,2022-06-30 15:29:00,0.243426,0.216455,28


In [49]:
market_df[market_df['nifty'].isna()]

,time,banknifty,nifty,tte


#### For each day, we identified the majority tte value and flagged all datapoints that differed from it, bringing unusual or inconsistent TTE values to our attention for further inspection.

In [50]:
market_df[market_df['tte'].isna()]

,time,banknifty,nifty,tte


In [51]:
def detect_tte_anomalies(df, time_col='time', tte_col='tte'):
    df = df.copy()
    df['Date'] = df[time_col].dt.date
    
    anomalies = pd.DataFrame()
    
    for day, group in df.groupby('Date'):
        # Find the majority TTE for the day
        majority_tte = group[tte_col].mode()
        if len(majority_tte) > 0:
            majority_tte = majority_tte[0]
        else:
            continue
        
        # Flag rows that differ from majority
        day_anomalies = group[group[tte_col] != majority_tte]
        anomalies = pd.concat([anomalies, day_anomalies])
    
    return anomalies

# Detect anomalies
tte_anomalies = detect_tte_anomalies(market_df, 'time', 'tte')
display(tte_anomalies)

,time,banknifty,nifty,tte,Date
34584,2021-01-25 09:39:00,0.312611,0.224372,3,2021-01-25
34585,2021-01-25 09:40:00,0.316106,0.229748,3,2021-01-25
34591,2021-01-25 09:46:00,0.308530,0.246817,3,2021-01-25
34592,2021-01-25 09:47:00,0.310240,0.246982,3,2021-01-25
34593,2021-01-25 09:48:00,0.293085,0.248248,3,2021-01-25
...,...,...,...,...,...
687564,2022-06-28 14:23:00,0.234324,0.222855,2,2022-06-28
687565,2022-06-28 14:24:00,0.234331,0.223002,2,2022-06-28
687566,2022-06-28 14:25:00,0.234336,0.222569,2,2022-06-28
687567,2022-06-28 14:26:00,0.234362,0.222513,2,2022-06-28


In [52]:
def fix_tte_anomalies(df, time_col='time', tte_col='tte'):
    df = df.copy()
    df['Date'] = df[time_col].dt.date
    
    for day, group in df.groupby('Date'):
        # Find the majority TTE for the day
        majority_tte = group[tte_col].mode()
        if len(majority_tte) > 0:
            majority_tte = majority_tte[0]
        else:
            continue
        
        # Replace anomalies with majority
        df.loc[group.index, tte_col] = group[tte_col].where(group[tte_col] == majority_tte, majority_tte)
    
    # Drop temporary Date column
    df.drop(columns=['Date'], inplace=True)
    
    return df

# Apply function
market_df = fix_tte_anomalies(market_df, time_col='time', tte_col='tte')

In [53]:
# Detect anomalies
tte_anomalies_new = detect_tte_anomalies(market_df, 'time', 'tte')
display(tte_anomalies_new)

,time,banknifty,nifty,tte,Date


### Now we have cleaned the data so let's proceed to check the basic statistical details of the data

In [54]:
cleaned_df = market_df.copy()

In [55]:
market_df.describe(include='all')

,time,banknifty,nifty,tte
count,180856,180856.000000,180856.000000,180856.000000
mean,2021-09-22 16:54:55.945945,0.252428,0.180532,16.151767
min,2021-01-01 09:15:00,0.152732,0.089355,2.000000
25%,2021-05-01 10:48:45,0.208084,0.141933,8.000000
50%,2021-08-29 12:22:30,0.246825,0.181463,16.000000
75%,2022-03-02 13:56:15,0.295293,0.217764,23.000000
max,2022-06-30 15:30:00,0.460619,0.306952,31.000000
std,NaN,0.058503,0.046373,8.713042


In [56]:
market_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180856 entries, 0 to 690511
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   time       180856 non-null  datetime64[us]
 1   banknifty  180856 non-null  float64       
 2   nifty      180856 non-null  float64       
 3   tte        180856 non-null  int32         
dtypes: datetime64[us](1), float64(2), int32(1)
memory usage: 10.2 MB


### During my previous internship, my manager, who had extensive experience in quantitative alpha research at Credit Suisse, shared a line that has stayed with me:

### "Quantitative research is not just about finding alphas or signals; the majority of the time should be spent on cleaning and preprocessing the data until it closely resembles real-world conditions, otherwise any alpha or signal you generate would be meaningless."

In [ ]:
market_df.to_excel("market_df.xlsx", index=False)